# Projeto 2 - Ciência dos Dados

##### Faculdade: Insper - Instituto de Ensino e Pesquisa

**Nome**: Jonas da Silva Lopes

**Nome**: William Augusto Reis da Silva

**Turma**: Engenharia | 2A


# Classificador automático de sentimento

Este projeto consiste na análise de dados de tweets, por meio da biblioteca tweepy, para realizar um classificador de sentimento, demonstrando se um tweety é relevante ou irrelevante. Para isso, realiamos algumas funções para que possamos determinar esta análise.

## Preparando o ambiente no jupyter:

Importando e instalando as bibliotecas

In [124]:
%%capture
!pip install tweepy
!pip install emoji

In [125]:
import tweepy
import emoji
import math
import os.path
import pandas as pd
import json
import re 
import functools
import operator
from random import shuffle

___
## Autenticando no  Twitter

* Conta utilizada: ***@William48253649***

In [126]:
# Identificador da conta no twitter: @William48253649

# Leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

# Configurando a biblioteca
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


O produto escolhido para a análise será a empresa Nubank, tendo em vista que é uma empresa em alta e que possui, possivelmente, diversas opiniões no Twitter

In [127]:
#Produto escolhido
produto = 'Nubank'

#Quantidade mínima de mensagens capturadas
n = 500
#Quantidade mínima de mensagens para a base de treinamento
t = 300

#Filtro de língua
lang = 'pt'

Capturando os dados do twitter:

In [128]:
# Cria um objeto para a captura
api = tweepy.API(auth)

# Inicia a captura
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower()) # Lower para deixar tudo minúsculo e facilitar a comparação
    i += 1
    if i > n:
        break

# Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [129]:
# Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    # Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    # Divide o conjunto de mensagens em duas planilhas
    # Planilha de treinamento
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])}).set()
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    # Planilha de teste
    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])}).set()
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    # Fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Após realizar a classificação manual das mensagens, como irrelevante (0) ou relevante (1), partimos para  a mudança desses valores para algo mais palpável para a análise. Com isso, fazemos a alteração que pode ser vista abaixo.

Mas, para realizar isso, primeiro foi necessário determinar critérios para a classificação:

- Mencionar o produto;
- A menção ao produto deve ser acompanhada de uma opinião;
- A opinião pode ser demonstrada na forma de indagações, reclamações, pode envolver sarcasmo, elogios e sugestões sobre serviços;
- A opinião afirmada deve ser clara;
- Emoctions também representam opiniões.

In [130]:
# Lendo o arquivo Nubank
mensagens = pd.read_excel("Nubank.xlsx")
mensagens.Relevância = mensagens.Relevância.astype('category')
mensagens.Relevância.cat.categories = ('Irrelevante', 'Relevante')


relevante = mensagens[mensagens.Relevância=="Relevante"]
irrelevante = mensagens[mensagens.Relevância=="Irrelevante"]

total = len(relevante) + len(irrelevante)

print("A quantidade de cada cada um é: \n\n", mensagens.Relevância.value_counts())

A quantidade de cada cada um é: 

 Irrelevante    184
Relevante      116
Name: Relevância, dtype: int64


In [131]:
'''A Partir dos dados obtidos, nota-se que:
        p(relevante) = 116/300
        p(irrelevante) = 184/300
'''
p_relev = 116/300*100
print("A probabilidade de ser relevante é", p_relev, "%")

p_irrelev = 184/300*100
print("A probabilidade de ser irrelevante é", p_irrelev, "%")

A probabilidade de ser relevante é 38.666666666666664 %
A probabilidade de ser irrelevante é 61.33333333333333 %


In [132]:
''' Função que troca pontuação por espaço '''
def cleanup(text):
    punctuation = '[!\-.:?;/,|@"\'()]'
    pattern = re.compile(punctuation)
    # Abaixo, determina que se troca por espaço
    text_subbed = re.sub(pattern, ' ', text)
    
    emoji_dividir = emoji.get_emoji_regexp().split(text_subbed)
    espaco_dividir = [substr.split() for substr in emoji_dividir]
    split = functools.reduce(operator.concat, espaco_dividir)
    
    return split

# Usando a função apply para fazer a limpeza nas mensagens
nubank_relev = relevante.Treinamento.apply(cleanup)
nubank_irrelev = irrelevante.Treinamento.apply(cleanup)

# Pegando as palavras da lista de Relevantes para contar
def pega_text(texto):
    # Pega as palavras para colocar numa lista
    lista = []
    i = 0
    for linha in texto:
        if len(linha) > 1:
            while i < len(linha):
                lista.append(linha[i])
                i += 1  
            i = 0
        elif len(linha) != 1 and (len(linha)-1) != 1:
            a = texto
            return a 
    return lista

### Frequência Absoluta

Relevantes

In [133]:
# Frequência dos Relevantes
lista_nubank_relev = pd.DataFrame(pega_text(nubank_relev))
lista_nubank_relev[0].value_counts()

nubank           122
o                 58
de                54
e                 52
eu                47
que               45
a                 35
meu               34
me                29
co                28
https             28
t                 28
um                27
não               26
é                 25
cartão            23
do                23
no                20
uma               16
q                 15
pra               15
na                15
com               14
to                14
só                13
limite            12
mais              12
nunca             11
tem               11
da                11
                ... 
deveria            1
pariu              1
cfo                1
valeriaales        1
melhores           1
eternidade         1
piso               1
moço               1
coisas             1
surpreenderam      1
392                1
desisto            1
seria              1
aguardando         1
500                1
blindadah          1
sites        

Irrelevantes

In [134]:
# Frequência dos Irrelevantes
lista_nubank_irrelev = pd.DataFrame(pega_text(nubank_irrelev))
lista_nubank_irrelev[0].value_counts()
lista_nubank_irrelev[0].value_counts()

nubank           154
o                 71
de                67
e                 62
que               60
https             58
co                58
t                 58
a                 57
não               52
é                 38
um                36
eu                34
com               33
da                33
do                31
se                30
gente             29
rt                28
tem               28
pra               26
meu               26
em                25
no                25
na                24
você              24
cartão            24
conta             22
uma               22
só                22
                ... 
mozão              1
aumentou           1
garantido          1
virtual”           1
msm                1
xaf1um2cqj         1
p94dsdayhw         1
sistema            1
lotéricas          1
faça               1
eoleite            1
✌🏽                 1
bora               1
conseguindo        1
vejo               1
bebada             1
piscina      

### Frequência Relativa

Relevante

In [135]:
lista_nubank_relev[0].value_counts(True)

nubank           0.056092
o                0.026667
de               0.024828
e                0.023908
eu               0.021609
que              0.020690
a                0.016092
meu              0.015632
me               0.013333
co               0.012874
https            0.012874
t                0.012874
um               0.012414
não              0.011954
é                0.011494
cartão           0.010575
do               0.010575
no               0.009195
uma              0.007356
q                0.006897
pra              0.006897
na               0.006897
com              0.006437
to               0.006437
só               0.005977
limite           0.005517
mais             0.005517
nunca            0.005057
tem              0.005057
da               0.005057
                   ...   
deveria          0.000460
pariu            0.000460
cfo              0.000460
valeriaales      0.000460
melhores         0.000460
eternidade       0.000460
piso             0.000460
moço        

Irrelevante

In [136]:
lista_nubank_irrelev[0].value_counts(True)

nubank           0.045441
o                0.020950
de               0.019770
e                0.018294
que              0.017704
https            0.017114
co               0.017114
t                0.017114
a                0.016819
não              0.015344
é                0.011213
um               0.010623
eu               0.010032
com              0.009737
da               0.009737
do               0.009147
se               0.008852
gente            0.008557
rt               0.008262
tem              0.008262
pra              0.007672
meu              0.007672
em               0.007377
no               0.007377
na               0.007082
você             0.007082
cartão           0.007082
conta            0.006492
uma              0.006492
só               0.006492
                   ...   
mozão            0.000295
aumentou         0.000295
garantido        0.000295
virtual”         0.000295
msm              0.000295
xaf1um2cqj       0.000295
p94dsdayhw       0.000295
sistema     

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, estamos ensinando nosso treinador a identificar as probabilidades, para posteriormente fazer a identificação e a leitura. 

#### Relevantes

In [143]:
'''Fazendo dicionários, para serem utilizados com objetivos na função'''

linhas_mens = mensagens.Treinamento.apply(cleanup)[0]

# Dicionário que guarda, dos relevantes, as palavras e quantas vezes apareceu
lista_counts_relevante = (lista_nubank_relev[0].value_counts()).to_dict()

# Dicionário que guarda a quantidade de relevantes e irrelevantes
dicionario_counts_relevancia = (mensagens.Relevância.value_counts()).to_dict()

In [144]:
# Função que calcula as probabilidade de ser relevante
def probabilidade_do_naive(lista_counts_relevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_relev):
    verossimilhança = 1
    for word in linhas_mens:
        for key, values in lista_counts_relevante.items():
            if key == word:
                verossimilhança *= (values+1)/(len(lista_nubank_relev)+len(linhas_mens))
    
    for rel, v in dicionario_counts_relevancia.items():
        if rel == "Relevante":
            total_rel = v
    priori = total_rel/total
    
    return (('{0:.2f}').format(math.log10(verossimilhança * priori)))

print(probabilidade_do_naive(lista_counts_relevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_relev))

-94.61


In [146]:
probs_relevante = [0]*300

i = 0
for h in probs_relevante:
    linhas_mens = mensagens.Treinamento.apply(cleanup)[i]
    probs_relevante[i] = probabilidade_do_naive(lista_counts_relevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_relev)
    i += 1
probs_relevante

['-94.61',
 '-53.49',
 '-11.68',
 '-15.12',
 '-15.80',
 '-38.27',
 '-18.61',
 '-46.45',
 '-22.97',
 '-38.04',
 '-16.53',
 '-23.09',
 '-49.49',
 '-36.68',
 '-58.67',
 '-25.11',
 '-17.16',
 '-45.70',
 '-15.93',
 '-62.63',
 '-38.78',
 '-18.45',
 '-65.70',
 '-59.38',
 '-25.30',
 '-114.36',
 '-44.16',
 '-17.44',
 '-31.07',
 '-9.52',
 '-72.67',
 '-69.88',
 '-18.44',
 '-48.00',
 '-87.51',
 '-5.95',
 '-22.36',
 '-33.87',
 '-43.83',
 '-27.18',
 '-42.00',
 '-25.25',
 '-21.77',
 '-14.23',
 '-13.21',
 '-71.81',
 '-26.51',
 '-24.71',
 '-43.45',
 '-14.01',
 '-17.75',
 '-13.90',
 '-20.44',
 '-21.03',
 '-14.98',
 '-18.41',
 '-52.00',
 '-40.51',
 '-52.59',
 '-77.81',
 '-91.71',
 '-33.68',
 '-16.24',
 '-36.92',
 '-24.38',
 '-27.66',
 '-34.76',
 '-37.50',
 '-38.78',
 '-55.44',
 '-1.66',
 '-20.56',
 '-42.84',
 '-1.66',
 '-27.87',
 '-22.04',
 '-3.85',
 '-63.79',
 '-40.09',
 '-13.65',
 '-20.23',
 '-17.12',
 '-10.49',
 '-124.87',
 '-17.97',
 '-14.15',
 '-19.16',
 '-46.28',
 '-38.78',
 '-34.81',
 '-11.99',
 '

#### Irrelevantes

In [147]:
# Dicionário que guarda, dos irrelevantes, as palavras e quantas vezes apareceu
lista_counts_irrelevante = (lista_nubank_irrelev[0].value_counts()).to_dict()

In [148]:
# Função que calcula as probabilidade de ser irrelevante
def probabilidade_do_naive_irr(lista_counts_irrelevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_irrelev):
    verossimilhança = 1
    # Identificando a quantidade e calculando a probabilidade de cada palavra do tweet
    for word in linhas_mens:
        for key, values in lista_counts_irrelevante.items():
            if key == word:
                verossimilhança *= (values+1) / (len(lista_nubank_irrelev)+len(linhas_mens))
    
    # Pegando a probabilidade e dividindo pelo total, usando Naive Bayes
    for rel, v in dicionario_counts_relevancia.items():
        if rel == "Irrelevante":
            total_rel = v
    priori = total_rel/total
    
    return (('{0:.2f}').format(math.log10(verossimilhança * priori)))

In [149]:
probs_irrelevante = [0]*300

i = 0
for h in probs_irrelevante:
    linhas_mens = mensagens.Treinamento.apply(cleanup)[i]
    probs_irrelevante[i] = probabilidade_do_naive_irr(lista_counts_irrelevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_irrelev)
    i += 1
#probs_irrelevante

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [150]:
df_teste = pd.ExcelFile("Nubank.xlsx")
df_teste1 = pd.read_excel(df_teste, 'Teste')

#### Criando os argumentos que entraram na função criada

Número de tweets relevantes e irrelevantes e seu total

In [151]:
df_teste1.Relevância = mensagens.Relevância.astype('category')
df_teste1.Relevância.cat.categories = ('Irrelevante', 'Relevante')

relevante_teste = df_teste1[df_teste1.Relevância=="Relevante"]
irrelevante_teste = df_teste1[df_teste1.Relevância=="Irrelevante"]

total_teste = len(relevante_teste) + len(irrelevante_teste)

print("A quantidade de cada cada um é: \n\n", df_teste1.Relevância.value_counts())
dicionario_counts_relevância1 = (df_teste1.Relevância.value_counts()).to_dict()
dicionario_counts_relevância1

A quantidade de cada cada um é: 

 Irrelevante    121
Relevante       79
Name: Relevância, dtype: int64


{'Irrelevante': 121, 'Relevante': 79}

#### Relevante

Frequência absoluta das palavras nos tweets relevantes e transformando em dicionário

In [152]:
# Usando a função apply para fazer a limpeza nas mensagens
df_teste2_rel = relevante_teste.Teste.apply(cleanup)
df_teste2_rel = pd.DataFrame(pega_text(df_teste2_rel))

dict_teste_counts_rel = (df_teste2_rel[0].value_counts()).to_dict()
total_teste_rel = df_teste2_rel

#### Irrelevante

Frequência absoluta das palavras nos tweets irrelevantes e tranformando em dicionário

In [153]:
df_teste2_irrel = pd.DataFrame(pega_text(irrelevante_teste.Teste.apply(cleanup)))
dict_teste_counts_irrel = (df_teste2_irrel[0].value_counts()).to_dict()
total_teste_irrel = df_teste2_irrel


In [154]:
linhas_mens_teste = df_teste1.Teste.apply(cleanup)[0]

In [155]:
print(probabilidade_do_naive_irr(dict_teste_counts_rel, dicionario_counts_relevância1, total, linhas_mens_teste, total_teste_rel))

-9.76


#### Relevante

In [156]:
probs_relevante2 = [0]*200

i = 0
for elemento in probs_relevante2:
    linhas_mens_teste = df_teste1.Teste.apply(cleanup)[i]
    probs_relevante2[i] = probabilidade_do_naive(dict_teste_counts_rel, dicionario_counts_relevância1, total_teste, linhas_mens_teste, df_teste2_rel)
    i += 1



#### Irrelevante

In [157]:
probs_irrelevante2 = [0]*200

i = 0
for elemento in probs_irrelevante2:
    linhas_mens_teste = df_teste1.Teste.apply(cleanup)[i]
    probs_irrelevante2[i] = probabilidade_do_naive_irr(dict_teste_counts_irrel, dicionario_counts_relevância1, total_teste, linhas_mens_teste, df_teste2_irrel)
    i += 1

#### Comparação

In [158]:
def comparation(relevante, irrelevante):
    i = 0 
    result = []
    for e in relevante:
        if relevante[i] > irrelevante[i]:
            result.append("Relevante")
        elif relevante[i] < irrelevante[i]:
            result.append("Irrelevante")
        # É muito improvável que ocorra. Porém, se ocorrer, consideramos que é irrelevante
        else:
            result.append(0)
        i += 1    
        
    return result

In [159]:
resultado_das_relevancias = comparation(probs_relevante2, probs_irrelevante2)

In [160]:
serie = []
for linha in df_teste1.Relevância:
    serie.append(linha)

def comparação_com_classificados(relevancias_comparativas, classificados):
    i = 0 
    true = 0
    false = 0
    comparação_final = []
    for e in classificados:
        if relevancias_comparativas[i] == classificados[i]:
            comparação_final.append(True)
            true += 1
        else:
            comparação_final.append(False)
            false += 1
        i+=1
            
    print(f'True: {true}, False: {false}')    
    
    return comparação_final

result = comparação_com_classificados(resultado_das_relevancias, serie)

True: 160, False: 40


In [161]:
serie
resultado_das_relevancias

def porcentagem(serie, resultado_das_relevancias):
    i = 0
    soma = 0
    ver_pos = 0
    fal_pos = 0
    ver_neg = 0
    fal_neg = 0
    
    for elemento in serie:
        if serie[i] == "Relevante" and resultado_das_relevancias[i]== "Relevante":
            ver_pos += 1
        elif serie[i] == "Irrelevante" and resultado_das_relevancias[i]== "Irrelevante":
            ver_neg += 1
        elif serie[i] == "Relevante" and resultado_das_relevancias[i]== "Irrelevante":
            fal_neg += 1
        elif serie[i] == "Irrelevante" and resultado_das_relevancias[i]== "Relevante":
            fal_pos += 1
        soma += 1
        i += 1
            
    return ver_pos/soma, ver_neg/soma, fal_pos/soma, fal_neg/soma

lista_porcentagem = porcentagem(serie, resultado_das_relevancias)
list_porc = []
i = 0
for e in lista_porcentagem:
    e *= 100
    list_porc.append(e)
    i += 1
    
    
print(f'Verdadeiros positivos: {list_porc[0]}%')
print(f'Falsos positivos: {list_porc[2]}%')
print(f'Verdadeiros negativos: {list_porc[1]}%')
print(f'Falsos negativos: {list_porc[3]}%')

Verdadeiros positivos: 32.0%
Falsos positivos: 12.5%
Verdadeiros negativos: 48.0%
Falsos negativos: 7.5%


___
### Concluindo

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**